In [1]:
#!pip install crewai crewai_tools langchain langchain_community gradio

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [3]:
search_tool = SerperDevTool()  
scrape_tool = ScrapeWebsiteTool()

In [4]:
import os
os.environ["SERPER_API_KEY"] = 'YOUR_API_KEY'

In [5]:
from langchain_community.chat_models import ChatOllama

In [6]:
#llm = ChatOllama(model="gemma2:latest", base_url="http://localhost:11434")
llm = ChatOllama(model="llama3:latest", base_url="http://localhost:11434")

In [7]:
# Agent 1: Idea Master
idea_master = Agent(
    role="Expert Idea Generator",
    goal="Produce three innovative, relevant, and actionable ideas to address "
         "specific opportunities related to the {topic}.",
    backstory=(
       "You are an expert idea generator having background in a diverse range"
       "of fields, such as a combination of engineering, science, and creative " 
       "You honed your skills through roles in innovation labs, startup ventures, " 
       "or research teams, where you learned to blend analytical thinking with "
       "creative problem-solving. You are particularly knowledgeable about the "
       "lastest developments around the {topic}."
    ),
    tools=[scrape_tool, search_tool],
    allow_delegation=False,
	llm=llm,
	verbose=True,
)

In [8]:
# Agent 2: Senior Technical Staff
technical_guy = Agent(
    role="Senior Technical Staff",
    goal="Using the three ideas generated by the idea generator in the previous step, "
         "describe the technical details of the development of innovative and scalable "
         "technology solutions for each one of them in separate paragraphs.",
    backstory=(
         "You are an expert technical staff member in a startup having a background "
         "characterized by a deep passion for technology and problem-solving, beginning "
         "with an engineering degree or equivalent experience. You gained hands-on "
         "experience at established tech companies or in innovative startups. Along "
         "the way, you have worked on high-impact projects, developed a knack for "
         "quickly adapting to new technologies, and demonstrated leadership in guiding "
         "teams through complex technical challenges. You are particularly knowledgeable " 
         "about the lastest technical developments around the {topic}."
    ),
    tools=[scrape_tool, search_tool],
    allow_delegation=False,
    llm=llm,
    verbose=True,
)

In [9]:
# Agent 3: Senior Product Manager
product_guy = Agent(
    role="Senior Product Manager",
    goal="Using the three ideas generated by the idea generator in the previous step, "
         "describe the details of product features, and vision and strategy for their "
         "developments in three separate paragraphs.",
    backstory=(
         "You are an expert senior product manager in a startup having a diverse background "
         "that includes a blend of formal education, such as an MBA or a degree in a related "
         "field, and hands-on experience in various product roles. You have started your career "
         "in a more technical or business-oriented position, gradually moving into product "
         "management by taking on roles that required close collaboration with engineering, "
         "marketing, and sales teams. Over the years, you have led multiple successful "
         "product launches, managed complex projects from conception through execution, and " 
         "developed a keen understanding of market dynamics and user needs. You are particularly "
         "knowledgeable about the lastest product developments around the {topic}."
    ),
    tools=[scrape_tool, search_tool],
    allow_delegation=False,
    llm=llm,
    verbose=True,
)

In [10]:
# Agent 4: Senior Business Intelligence Analyst
business_guy = Agent(
    role="Senior Business Intelligence Analyst",
    goal="Using the three ideas generated by the idea generator in the previous step, for each "
         "one of them in separate paragraphs describe in details what kind of tasks you should "
         "do to provide strategic insights and actionable recommendations to guide data-driven "
         "decision-making to support the company's growth and operational efficiency.",
    backstory=(
        "You are an expert senior business intelligence analyst having a rich background that "
        "began with a degree in data science or statistics, followed by hands-on experience "
        "in various analytical roles across different industries. Early in your career, you "
        "have worked with large datasets and developed their skills in data visualization and "
        "reporting at established companies, where they contributed to critical projects and "
        "gained a deep understanding of business operations. You are particularly knowledgeable "
        "about the lastest business developments around the {topic}."
    ),
    tools=[scrape_tool, search_tool],   
    allow_delegation=False,
	llm=llm,
	verbose=True,
)

In [11]:
# Task 1: Task for Idea Master
idea_task = Task(
    description=(
        "Conduct a thorough research about {topic} and come up with three interesting "
        "ideas related to the {topic}. Make sure you find any interesting and relevant "
        "information given the current year is 2024."
    ),
    expected_output=(
        "A list with three bullet points of the most interesting ideas related to the "
        "{topic}."
    ),
    agent=idea_master,
	output_file='idea_report.md',
)

In [12]:
# Task 2: Task for Senior Technical Staff
technical_task = Task(
    description=(
        "Conduct a thorough research about the three ideas provided by the idea generator "
        "in the previous step and write the technical details of the solutions to build a "
        "product for each one of the ideas in separate paragraphs each one consisted of a "
        "few sentences. Make sure you find any interesting and relevant information given "
        "the current year is 2024."
    ),
    expected_output=(
        "A three paragraphs each one consisted of a few sentences describing the technical "
        "details related to each product, plus a file with the same contents formatted as "
        "markdown without '```'. "
    ),
    agent=technical_guy,
    context=[idea_task],
	output_file='technical_report.md',
)

In [13]:
# Task 3: Task for Senior Product Manager
product_task = Task(
    description=(
        "Conduct a thorough research about the three ideas provided by the idea generator " 
        "in the previous step and write the details of product features for each one of "
        "the ideas in separate paragraphs each one consisted of a few sentences. Make sure "
        "you find any interesting and relevant information given the current year is 2024."
    ),
    expected_output=(
        "A three paragraphs each one consisted of a few sentences describing the details of "
        "the features of the three products, plus a file with the same contents formatted as "
        "markdown without '```'."
    ),
    agent=product_guy,
    context=[idea_task],
	output_file='product_report.md',
)

In [14]:
# Task 4: Task for Senior Business Intelligence Analyst
business_task = Task(
    description=(
        "Conduct a thorough research about the three ideas provided by the idea generator "
        "in the previous step and write the details of the steps which are required to be "
        "taken the business intelligence analyst to make sure the continuous growth of "
        "the company. Make sure you find any interesting and relevant information given "
        "the current year is 2024."
    ),
    expected_output=(
        "A three paragraphs each one consisted of a few sentences describing the required "
        "works should be done by the business intelligence analyst to make sure the "
        "continuous growth of the company, plus a file with the same contents formatted "
        "as markdown without '```'."
    ),
    agent=business_guy,
    context=[idea_task],
	output_file='business_report.md',
)

In [15]:
startup_brainstorming = Crew(
    agents=[idea_master,
            technical_guy,
            product_guy,
            business_guy],
    tasks=[idea_task,
           technical_task,
           product_task,
           business_task],
    process=Process.sequential,
    verbose=True
)

In [16]:
import gradio as gr
from IPython.display import Markdown

/Users/asataryd/miniforge3/envs/startup-brainstorming/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def crew_run(inputs):
    startup_brainstorming.kickoff(inputs={'topic': inputs})

def markdown_read(file):
    return Markdown(file).data

def show_all():
    return "", "./idea_report.md", "./technical_report.md", "./product_report.md", "./business_report.md"

In [18]:
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    with gr.Row():
        # Choose a Topic
        text_topic = gr.Textbox(label="The Initial Topic", interactive=True, placeholder="Select a Topic!")
        btn_start = gr.Button("Start Brainstorming")
        
    with gr.Column():
        # Ideas
        text_idea = gr.Textbox(label="Startup Ideas", interactive=False, placeholder="./idea_report.md")
        
        # Technical & Product & Business Stuff
        with gr.Row():
          text_technical = gr.Textbox(label="Technical Stuff", interactive=False, placeholder="./technical_report.md")      
          text_product = gr.Textbox(label="Product Stuff", interactive=False, placeholder="./product_report.md")
          text_business = gr.Textbox(label="Business Stuff", interactive=False, placeholder="./business_report.md")

        # Clear All
        btn_show = gr.Button("Show All")
    
    btn_start.click(crew_run, inputs=[text_topic], queue=True)
        
    text_idea.change(fn=markdown_read, inputs=[text_idea], outputs=[text_idea], queue=True)
    text_technical.change(fn=markdown_read, inputs=[text_technical], outputs=[text_technical], queue=True)
    text_product.change(fn=markdown_read, inputs=[text_product], outputs=[text_product], queue=True)
    text_business.change(fn=markdown_read, inputs=[text_business], outputs=[text_business], queue=True)
    
    btn_show.click(show_all, outputs=[text_topic, text_idea, text_technical, text_product, text_business], queue=True)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d92475da9333e5f270.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 [2024-09-17 16:45:49][DEBUG]: == Working Agent: Expert Idea Generator
 [2024-09-17 16:45:49][INFO]: == Starting Task: Conduct a thorough research about Neurotechnology and come up with three interesting ideas related to the Neurotechnology. Make sure you find any interesting and relevant information given the current year is 2024.


> Entering new CrewAgentExecutor chain...
Thought: I should start by researching the current state of Neurotechnology in 2024.

Action: Read website content
Action Input: {'website_url': 'https://neurotechnology.org/neurotech-2024/' 



Let's get started!

Thought:
I should start by researching the current state of Neurotechnology in 2024.

Action: Read website content
Action Input: {'website_url': 'https://neurotechnology.org/neurotech-2024/' 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Let's get started!

Thought:
I should start by researching the current state of Neurotechnology in 2024.

Act